<a href="https://colab.research.google.com/github/s20542/skin_disease/blob/main/skin_disease_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"eleeas","key":"a18fd1cb6a8ea64e024b8502b310e1a9"}'}

In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d shubhamgoel27/dermnet

Dataset URL: https://www.kaggle.com/datasets/shubhamgoel27/dermnet
License(s): Attribution-NonCommercial-NoDerivatives 4.0 International (CC BY-NC-ND 4.0)
 99% 1.71G/1.72G [00:07<00:00, 241MB/s]
100% 1.72G/1.72G [00:07<00:00, 240MB/s]


In [ ]:
!unzip dermnet.zip


Streaming output truncated to the last 5000 lines.
  inflating: train/Seborrheic Keratoses and other Benign Tumors/keloids-8.jpg  
  inflating: train/Seborrheic Keratoses and other Benign Tumors/keloids-80.jpg  
  inflating: train/Seborrheic Keratoses and other Benign Tumors/keloids-9.jpg  
  inflating: train/Seborrheic Keratoses and other Benign Tumors/keratoacanthoma-100.jpg  
  inflating: train/Seborrheic Keratoses and other Benign Tumors/keratoacanthoma-101.jpg  
  inflating: train/Seborrheic Keratoses and other Benign Tumors/keratoacanthoma-102.jpg  
  inflating: train/Seborrheic Keratoses and other Benign Tumors/keratoacanthoma-103.jpg  
  inflating: train/Seborrheic Keratoses and other Benign Tumors/keratoacanthoma-104.jpg  
  inflating: train/Seborrheic Keratoses and other Benign Tumors/keratoacanthoma-105.jpg  
  inflating: train/Seborrheic Keratoses and other Benign Tumors/keratoacanthoma-106.jpg  
  inflating: train/Seborrheic Keratoses and other Benign Tumors/keratoacanthom

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
import cv2
import random
from tensorflow.keras.optimizers import SGD

In [ ]:
resnet = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
resnet.trainable=False

In [ ]:
from tensorflow.keras.applications import ResNet50
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers

# Set constants
batch_size = 64
image_size = 224
train_path = "/content/train"
test_path = "/content/test"
num_classes = 23

# Create the MirroredStrategy instance for using multiple GPUs
strategy = tf.distribute.MirroredStrategy()

def train_val_generators(train_dir, val_dir, image_size, batch_size):
    train_datagen = ImageDataGenerator(
        rescale=1./255, #range [0 1]
        shear_range=0.2,
        zoom_range=0.3,
        width_shift_range=0.2,
        height_shift_range=0.2,
        brightness_range=[0.2, 1.2],
        rotation_range=0.2,
        horizontal_flip=True
    )

    train_generator = train_datagen.flow_from_directory(
        directory=train_dir,
        batch_size=batch_size,
        class_mode='categorical',
        target_size=(image_size, image_size)
    )

    test_datagen = ImageDataGenerator(rescale=1./255)

    test_generator = test_datagen.flow_from_directory(
        directory=val_dir,
        batch_size=batch_size,
        class_mode='categorical',
        target_size=(image_size, image_size)
    )

    return train_generator, test_generator

# Initialize the data generators
train_generator, test_generator = train_val_generators(train_path, test_path, image_size, batch_size)
class_names = train_generator.class_indices

# Define the model within the strategy scope
with strategy.scope():
    base_model = ResNet50(
        input_shape=(image_size, image_size, 3),  # Define the input shape
        include_top=False,                       # Exclude the fully connected top layers
        weights='imagenet'                       # Use pretrained weights from ImageNet
    )

    # Make all layers trainable
    for layer in base_model.layers:
        layer.trainable = True

    def output_of_last_layer(pre_trained_model, limit_layer):
        last_desired_layer = pre_trained_model.get_layer(limit_layer)
        print('last layer output shape: ', last_desired_layer.output.shape)
        last_output = last_desired_layer.output
        print('last layer output: ', last_output)
        return last_output

    # Update the layer name to the correct one for ResNet50
    last_output = output_of_last_layer(base_model, 'conv5_block3_out')

    # Add custom layers
    x = tf.keras.layers.Flatten()(last_output)
    x = tf.keras.layers.Dense(1024, activation='relu', kernel_regularizer=regularizers.l2(0.01))(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(0.3)(x)
    x = tf.keras.layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.01))(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(0.3)(x)
    x = tf.keras.layers.Dense(23, activation='softmax')(x)

    model = tf.keras.models.Model(base_model.input, x)

    # Compile the model
    model.compile(
        optimizer=Adam(learning_rate=0.0001),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

from tensorflow.keras.callbacks import EarlyStopping

# Define early stopping callback
custom_early_stopping = EarlyStopping(
    monitor='val_accuracy',
    patience=30,
    min_delta=0.001,
    mode='min'
)
# Train the model
history = model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=45,
    callbacks=[custom_early_stopping]
)

# Save the model to the working directory
model.save('/content/model.h5')

Found 15557 images belonging to 23 classes.
Found 4002 images belonging to 23 classes.
last layer output shape:  (None, 7, 7, 2048)
last layer output:  <KerasTensor shape=(None, 7, 7, 2048), dtype=float32, sparse=False, name=keras_tensor_896>


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/45
244/244 ━━━━━━━━━━━━━━━━━━━━ 262s 946ms/step - accuracy: 0.1764 - loss: 29.3847 - val_accuracy: 0.0262 - val_loss: 26.1659
Epoch 2/45
244/244 ━━━━━━━━━━━━━━━━━━━━ 236s 939ms/step - accuracy: 0.3083 - loss: 22.6913 - val_accuracy: 0.0545 - val_loss: 19.7307
Epoch 3/45
244/244 ━━━━━━━━━━━━━━━━━━━━ 235s 937ms/step - accuracy: 0.3765 - loss: 17.2245 - val_accuracy: 0.1324 - val_loss: 15.6264
Epoch 4/45
244/244 ━━━━━━━━━━━━━━━━━━━━ 237s 943ms/step - accuracy: 0.4337 - loss: 13.2977 - val_accuracy: 0.2946 - val_loss: 11.8376
Epoch 5/45
244/244 ━━━━━━━━━━━━━━━━━━━━ 236s 941ms/step - accuracy: 0.4736 - loss: 10.5117 - val_accuracy: 0.4098 - val_loss: 9.5318
Epoch 6/45
244/244 ━━━━━━━━━━━━━━━━━━━━ 235s 937ms/step - accuracy: 0.5163 - loss: 8.5743 - val_accuracy: 0.3683 - val_loss: 8.3225
Epoch 7/45
244/244 ━━━━━━━━━━━━━━━━━━━━ 233s 930ms/step - accuracy: 0.5284 - loss: 7.2770 - val_accuracy: 0.4760 - val_loss: 6.8244
Epoch 8/45
244/244 ━━━━━━━━━━━━━━━━━━━━ 236s 939ms/step - accuracy: